<a href="https://colab.research.google.com/github/niatullah/myproject/blob/main/PersianPhotoToWord.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# ===== 1) نصب وابستگی‌ها =====
!pip -q install easyocr==1.7.1 python-docx>=1.0.0 scikit-image>=0.22.0 pillow>=10.0.0 opencv-python>=4.9.0 matplotlib>=3.8.0
# برای تشخیص فرمول‌ها:
!pip -q install pix2tex==0.1.3

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 427.0/427.0 kB 23.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 431.5/431.5 kB 32.8 MB/s eta 0:00:00


In [3]:
import cv2, sys, os, io, re
import numpy as np
import easyocr
from skimage.filters import threshold_sauvola
from docx import Document
from docx.shared import Pt, Inches
from docx.enum.text import WD_ALIGN_PARAGRAPH
from docx.oxml import OxmlElement
from docx.oxml.ns import qn
import matplotlib.pyplot as plt
from PIL import Image

try:
    from pix2tex.cli import LatexOCR
    _pix2tex_available = True
except Exception as e:
    print('⚠️ pix2tex load warning:', e)
    _pix2tex_available = False

print('✅ نصب و import انجام شد. pix2tex:', _pix2tex_available)

⚠️ pix2tex load warning: 1 validation error for InitSchema
std_range
  Input should be a valid tuple [type=tuple_type, input_value=10, input_type=int]
    For further information visit https://errors.pydantic.dev/2.11/v/tuple_type
✅ نصب و import انجام شد. pix2tex: False


/usr/local/lib/python3.11/dist-packages/albumentations/core/validation.py:114: UserWarning: ShiftScaleRotate is a special case of Affine transform. Please use Affine transform instead.
  original_init(self, **validated_kwargs)
/usr/local/lib/python3.11/dist-packages/pix2tex/dataset/transforms.py:7: UserWarning: Argument(s) 'value' are not valid for transform ShiftScaleRotate
  [alb.ShiftScaleRotate(shift_limit=0, scale_limit=(-.15, 0), rotate_limit=1, border_mode=0, interpolation=3,
/usr/local/lib/python3.11/dist-packages/pix2tex/dataset/transforms.py:9: UserWarning: Argument(s) 'value' are not valid for transform GridDistortion
  alb.GridDistortion(distort_limit=0.1, border_mode=0, interpolation=3, value=[255, 255, 255], p=.5)], p=.15),


In [4]:
# ===== 2) توابع: پیش‌پردازش، تشخیص متن/فرمول، ساخت ورد =====
import torch
import cv2, numpy as np, io
from skimage.filters import threshold_sauvola

# --- نرمال‌سازی حروف فارسی ---
ARABIC_YEH='\u064A'; PERSIAN_YEH='\u06CC'
ARABIC_KEHEH='\u0643'; PERSIAN_KEHEH='\u06A9'
TATWEEL='\u0640'

def normalize_persian(s:str)->str:
    if not s: return s
    s = s.replace(ARABIC_YEH, PERSIAN_YEH).replace(ARABIC_KEHEH, PERSIAN_KEHEH)
    s = s.replace(TATWEEL,'')
    s = ' '.join(s.split())
    return s

# --- پیش‌پردازش تصویر ---
def to_gray(img):
    return cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) if img.ndim==3 else img

def denoise(gray):
    # فیلتر دوطرفه: حذف نویز با حفظ لبه‌ها (برای دست‌نویس مناسب است)
    return cv2.bilateralFilter(gray, 9, 75, 75)

def enhance_contrast(gray):
    # افزایش کنتراست محلی
    clahe = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(8,8))
    return clahe.apply(gray)

def binarize_sauvola(gray):
    # آستانه‌گذاری سازوولا برای نور ناهمگن
    win = max(25, min((gray.shape[0]//20)*2+1, 151))
    thresh = threshold_sauvola(gray, window_size=win, k=0.2)
    bin_ = (gray > thresh).astype(np.uint8)*255
    # در صورت نیاز معکوس کن تا متن مشکی روی پس زمینه سفید شود
    if np.mean(bin_) < 127:
        bin_ = 255 - bin_
    return bin_

def deskew(binary):
    # صاف‌کردن کجی خطوط با استفاده از minAreaRect
    coords = np.column_stack(np.where(binary==0))
    if coords.size==0:
        return binary
    rect = cv2.minAreaRect(coords.astype(np.float32))
    ang = rect[-1]
    ang = -(90+ang) if ang < -45 else -ang
    (h,w)=binary.shape[:2]
    M=cv2.getRotationMatrix2D((w//2,h//2), ang, 1.0)
    return cv2.warpAffine(binary, M, (w,h), flags=cv2.INTER_CUBIC, borderMode=cv2.BORDER_REPLICATE)

def preprocess(img_bgr):
    g=to_gray(img_bgr)
    g=denoise(g)
    g=enhance_contrast(g)
    b=binarize_sauvola(g)
    b=deskew(b)
    return b

# --- OCR سازها ---
def build_readers(enable_formulas=True):
    gpu = torch.cuda.is_available()
    txt_reader = easyocr.Reader(['fa','ar','en'], gpu=gpu)
    formula_reader = None
    if enable_formulas and _pix2tex_available:
        try:
            formula_reader = LatexOCR()
        except Exception as e:
            print('⚠️ init pix2tex:', e)
            formula_reader = None
    return txt_reader, formula_reader

def crop_bbox(img, box):
    # box: [[x1,y1], [x2,y2], [x3,y3], [x4,y4]]
    x_min = max(0, int(min(p[0] for p in box)))
    y_min = max(0, int(min(p[1] for p in box)))
    x_max = min(img.shape[1], int(max(p[0] for p in box)))
    y_max = min(img.shape[0], int(max(p[1] for p in box)))
    return img[y_min:y_max, x_min:x_max].copy()

MATH_CHARS = set(list("+-=×÷/*()[]{}^_<>|%~≈≠≤≥∞∑∏√∫±:;,.0123456789"))

def is_probably_formula(text:str, conf:float)->bool:
    if not text: return False
    hits = sum(1 for ch in text if ch in MATH_CHARS)
    ratio = hits / max(1, len(text))
    # اگر نمادهای ریاضی زیاد بود یا اعتماد متن پایین بود، فرمول فرض می‌کنیم
    return (ratio > 0.25) or (conf < 0.25)

def ocr_blocks(img_bgr, txt_reader, formula_reader=None, min_conf=0.2):
    proc = preprocess(img_bgr)
    rgb = cv2.cvtColor(proc, cv2.COLOR_GRAY2RGB)
    results = txt_reader.readtext(rgb, detail=1, paragraph=False)
    blocks = []
    for r in results:
        # EasyOCR: [box, text, conf]
        try:
            box, txt, conf = r
        except Exception:
            if isinstance(r, (list, tuple)) and len(r) >= 2:
                txt = r[1]
                conf = r[2] if len(r) > 2 else 0.0
                box = r[0] if len(r) > 0 else None
            else:
                continue
        conf = conf or 0.0
        if not box:
            continue

        crop = crop_bbox(img_bgr, box)
        # تصمیم متن/فرمول
        if (formula_reader is not None) and is_probably_formula(txt, conf):
            try:
                pil = Image.fromarray(cv2.cvtColor(crop, cv2.COLOR_BGR2RGB))
                latex = formula_reader(pil)
                kind = 'formula'
                value = (latex or '').strip()
                if not value:
                    # اگر چیزی نداد، به متن برگرد
                    kind = 'text'
                    value = normalize_persian(txt)
            except Exception as e:
                kind = 'text'
                value = normalize_persian(txt)
        else:
            kind = 'text'
            value = normalize_persian(txt)

        try:
            y = min(p[1] for p in box); x = min(p[0] for p in box)
        except Exception:
            y, x = 0, 0

        blocks.append({'y':y, 'x':x, 'kind':kind, 'value':value, 'crop':crop})

    # مرتب‌سازی: از بالا به پایین، و در هر خط راست به چپ
    blocks.sort(key=lambda b: (b['y'], -b['x']))
    return blocks

# --- رندر فرمول به تصویر با matplotlib (بدون نیاز به LaTeX سیستم) ---
def latex_to_png_bytes(latex:str, dpi=300):
    fig = plt.figure(figsize=(0.01,0.01), dpi=dpi)
    fig.text(0.5, 0.5, f'$ {latex} $', ha='center', va='center')
    buf = io.BytesIO()
    plt.axis('off')
    plt.savefig(buf, format='png', bbox_inches='tight', pad_inches=0.1)
    plt.close(fig)
    buf.seek(0)
    return buf.read()

# --- ساخت فایل ورد راست‌به‌چپ ---
def add_bidi(paragraph):
    p = paragraph._p
    pPr = p.get_or_add_pPr()
    bidi = OxmlElement('w:bidi'); bidi.set(qn('w:val'), '1'); pPr.append(bidi)

def write_docx(blocks, out_path='output.docx', font_name='Arial', font_size=12, eq_width_in=2.2):
    doc = Document()
    style = doc.styles['Normal']
    style.font.name = font_name
    # برای پشتیبانی بهتر از اسکریپت‌های پیچیده
    style._element.rPr.rFonts.set(qn('w:eastAsia'), font_name)
    style.font.size = Pt(font_size)

    for b in blocks:
        if b['kind'] == 'text':
            p = doc.add_paragraph()
            add_bidi(p)
            p.alignment = WD_ALIGN_PARAGRAPH.RIGHT
            p.add_run(b['value'])
        elif b['kind'] == 'formula':
            try:
                png = latex_to_png_bytes(b['value'])
                tmp = io.BytesIO(png)
                p = doc.add_paragraph()
                p.alignment = WD_ALIGN_PARAGRAPH.CENTER
                p.add_run().add_picture(tmp, width=Inches(eq_width_in))
            except Exception:
                # اگر رندر شکست خورد، خود متن LaTeX را درج کن
                p = doc.add_paragraph()
                add_bidi(p)
                p.alignment = WD_ALIGN_PARAGRAPH.RIGHT
                p.add_run(b['value'])

    doc.save(out_path)
    return out_path

print('✅ توابع آماده‌اند.')

✅ توابع آماده‌اند.


In [6]:
# ===== 3) آپلود، اجرا و دانلود ورد =====
from google.colab import files

# اگر فرمول نمی‌خواهی، این را False بگذار:
ENABLE_FORMULAS = True

uploaded = files.upload()  # چند تصویر دست‌نویس انتخاب کن (jpg/png/jpeg/tif...)
paths = list(uploaded.keys())
if not paths:
    raise SystemExit('هیچ تصویری انتخاب نشد.')

txt_reader, formula_reader = build_readers(enable_formulas=ENABLE_FORMULAS)

all_blocks = []
for i, pth in enumerate(sorted(paths)):
    print('▶️ پردازش:', pth)
    img = cv2.imread(pth)
    if img is None:
        print('⚠️ خواندن تصویر ناموفق:', pth)
        continue
    blocks = ocr_blocks(img, txt_reader, formula_reader, min_conf=0.2)
    if i > 0:
        # جداکننده صفحه بین تصاویر مختلف (یک خط خالی)
        all_blocks.append({'kind':'text','value':'','y':999999, 'x':0, 'crop':None})
    all_blocks.extend(blocks)

docx_path = write_docx(all_blocks, out_path='output.docx', font_name='Arial', font_size=12, eq_width_in=2.2)
print('📄 ذخیره شد:', docx_path)
files.download(docx_path)

KeyboardInterrupt: 